In [1]:
import numpy as np
import torch
import json
import sys
import pandas as pd 
import os 

/home2/shivprasad.sagare/miniconda3/envs/xalign_role/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!gdown --id 1VAMOTosRNcXJh_4XxpUEuB09G4AwOauT
!unzip plan_order.zip

/home2/shivprasad.sagare/miniconda3/envs/xalign_role/lib/python3.7/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1VAMOTosRNcXJh_4XxpUEuB09G4AwOauT 

unzip:  cannot find or open plan_order.zip, plan_order.zip.zip or plan_order.zip.ZIP.


In [2]:
lang='en'

def load_jsonl(file_path):
    res = []
    with open(file_path, encoding='utf-8') as dfile:
        for line in dfile.readlines():
            res.append(json.loads(line.strip()))
    return res


filename = "{lang}_order.jsonl".format(lang=lang)

reader  = load_jsonl(filename)


jsonl_data = pd.DataFrame(reader)


In [3]:

jsonl_data.to_csv('en_order.csv', sep='\t')

In [3]:
from torch import cuda
device = "cuda" if cuda.is_available() else "cpu"

In [4]:
from torch.utils.data import Dataset, DataLoader


In [ ]:

class JsonL(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, isTarget = True):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.targ = isTarget

    def process_facts_shiva(facts, entity):
        """ linearizes the facts on the encoder side """
        triples = []
        subject = entity
        # triples.append([' <S> '+subject])

        for triple in facts:
            predicate = triple[0]
            object = triple[1]
            triples.append([' S| '+subject, ' P| '+predicate, ' O| '+object])

            for quals in triple[2]:
                qual_subject = object
                qual_predicate = quals[0]
                qual_object = quals[1]
                triples.append([' S| '+qual_subject, ' P| '+qual_predicate, ' O| '+qual_object])

        return triples

    def __getitem__(self, index):

        sent  = self.data.sentence[index]
        factlist = self.data.facts[index]
        entity = self.data.entity_name[index]
        section = self.data.native_sentence_section[index]
        lang = self.data.lang[index]
    
        fact_str = self.process_facts_shiva(factlist,entity)
        
#         inputs = self.tokenizer.encode_plus(
#             fact_str,
#             None,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             pad_to_max_length=True,
#             return_token_type_ids=True,
#             truncation=True,
#         )
#         CS_ids = inputs["input_ids"]
#         CS_mask = inputs["attention_mask"]
        
        
        
        if self.targ :
            ref_order = self.data.order[index] 

            return {
                "fact_str" : fact_str,
                "sent" : sent,
                "ref_order" : ref_order,
                "lang" : lang
            }
        else :
            return {
                "fact_str" : fact_str,
                "sent" : sent,
                "lang" : lang
            }
            
            

    def __len__(self):
        return self.len





In [7]:
def collate(batch):
    
    fact_str_list = []
    sent_list = []
    ref_list = []
    lang_list = []
    
    ref_flag = True 
    
    try :
        k = batch[0]['ref_order']
    except:
        ref_flag = False
    
    for i in batch :
        fact_str_list.append(i['fact_str'])
        sent_list.append(i['sent'])
        lang_list.append(i['lang'])
        
        
        if ref_flag :
            ref_list.append(i['ref_order'])
        
    
    if ref_flag :
        return {'fact_str':fact_str_list,'sent' : sent_list,'ref_order':ref_list, 'lang': lang_list}
    else:
        return {'fact_str':fact_str_list,'sent' : sent_list, 'lang': lang_list}
    
    

In [5]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel


In [ ]:

model_name = "google/mt5-small" #google/muril-base-cased, bert-base-multilingual-cased, xlm-roberta-base, sentence-transformers/LaBSE

tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir="/scratch")
model = AutoModel.from_pretrained(model_name,output_hidden_states=True, cache_dir="/scratch")

sent = "Today is a weird day, let's see if it works "
facts = "let's see if it works "

decoded = tokenizer.encode_plus(sent, return_tensors="pt")
encoded = tokenizer.encode_plus(facts, return_tensors="pt")

with torch.no_grad():
    output = model(input_ids=encoded["input_ids"], decoder_input_ids=decoded["input_ids"],output_attentions=True)

print (output)

In [7]:
print ("len encoder hiddenstates", len(output.encoder_hidden_states))
print ("len encoder input tokens", len(encoded["input_ids"][0]))
print ("len decocder hidden states", len(output.decoder_hidden_states))
print ("len decoder input tokens",len(decoded["input_ids"][0]))
print ("cross attention", output.cross_attentions[-1][0].shape)
print ("decoder attention",output.decoder_attentions[-1][0].shape)
print ("encoder attention",output.encoder_attentions[-1].shape)


len encoder hiddenstates 9
len encoder input tokens 9
len decocder hidden states 9
len decoder input tokens 17
cross attention torch.Size([6, 17, 9])
decoder attention torch.Size([6, 17, 17])
encoder attention torch.Size([1, 6, 9, 9])


In [6]:

tokenizer = AutoTokenizer.from_pretrained('google/mt5-small',cache_dir="/scratch")


new_tokens = ['S|', 'P|', 'O|']
new_tokens_vocab = {}
new_tokens_vocab['additional_special_tokens'] = []
for idx, t in enumerate(new_tokens):
    new_tokens_vocab['additional_special_tokens'].append(t)
num_added_toks = tokenizer.add_special_tokens(new_tokens_vocab)
# self.config_args.logger.critical('added %s tokens' % num_added_toks)

In [7]:
tokenizer = AutoTokenizer.from_pretrained('google/mt5-small')
tokenizer.add_tokens(['S|', 'P|', 'O|'])

Downloading: 100%|██████████| 82.0/82.0 [00:00<00:00, 39.9kB/s]
Downloading: 100%|██████████| 4.11M/4.11M [00:04<00:00, 1.07MB/s]
Downloading: 100%|██████████| 99.0/99.0 [00:00<00:00, 72.1kB/s]


3

In [1]:
!export WANDB_API_KEY=bf6eddaca0cddb4d9e70aa37fb5ef56202d7ef74

import wandb

run = wandb.init()

artifact = run.use_artifact('shivprasad/xalign/model-2w17e6wd:v0', type='model')

artifact_dir = artifact.download('./')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shivprasad. Use `wandb login --relogin` to force relogin
/home2/shivprasad.sagare/miniconda3/envs/xalign_role/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


wandb: Downloading large artifact model-2w17e6wd:v0, 3444.44MB. 1 files... Done. 0:4:23.3


In [8]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel

model_name = "google/mt5-small" #google/muril-base-cased, bert-base-multilingual-cased, xlm-roberta-base, sentence-transformers/LaBSE
PATH = "model.ckpt" 

model = AutoModel.from_pretrained(model_name,output_hidden_states=True)

model.resize_token_embeddings(len(tokenizer))

checkpoint = torch.load(PATH)
print (checkpoint.keys())

new_state_dict = {}
for ij in checkpoint['state_dict']:
    #     print (ij[:6])
    if ij[:6] == 'model.':
        newkey = ij[6:]
        new_state_dict[newkey] = checkpoint['state_dict'][ij]
    else :
        print (ij)
    
del new_state_dict['lm_head.weight']
model.load_state_dict(new_state_dict)

Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MT5Model were not initialized from the model checkpoint at google/mt5-small and are newly initialized: ['encoder.role_embed.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])


<All keys matched successfully>

In [12]:
import unidecode 

languages_map = {
    'en': {"label": "English", 'id': 0},
    'hi': {"label": "Hindi", 'id': 1},
    'te': {"label": "Telugu", 'id': 2},
    'bn': {"label": "Bengali", 'id': 3},
    'pa': {"label": "Punjabi", 'id': 4},
    'ur': {"label": "Urdu", 'id': 5},
    'or': {"label": "Odia", 'id': 6},
    'as': {"label": "Assamese", 'id': 7},
    'gu': {"label": "Gujarati", 'id': 8},
    'mr': {"label": "Marathi", 'id': 9},
    'kn': {"label": "Kannada", 'id': 10},
    'ta': {"label": "Tamil", 'id': 11},
    'ml': {"label": "Malayalam", 'id': 12}
}


def get_nodes(n):
    n = n.strip()
    n = n.replace('(', '')
    n = n.replace('\"', '')
    n = n.replace(')', '')
    n = n.replace(',', ' ')
    n = n.replace('_', ' ')
    n = unidecode.unidecode(n)
    return n


def get_relation(n):
    n = n.replace('(', '')
    n = n.replace(')', '')
    n = n.strip()
    n = n.split()
    n = "_".join(n)
    return n


def linear_fact_str(fact, enable_qualifiers=True):
    fact_str = ['<R>', get_relation(fact[0]).lower(), '<T>', get_nodes(fact[1]).lower()]
    qualifier_str = [' '.join(['<QR>', get_relation(x[0]).lower(), '<QT>', get_nodes(x[1]).lower()]) for x in fact[2]]
    if enable_qualifiers and len(fact[2])>0:
        fact_str += [' '.join(qualifier_str)]
    return fact_str

def process_facts(facts, entity, section, language):
    """ linearizes the facts on the encoder side """
    linearized_facts = []
    for i in range(len(facts)):
        linearized_facts += linear_fact_str(facts[i], enable_qualifiers=True)
    processed_facts_str = ' '.join(linearized_facts)
    pre_string = "<H> %s %s <S> %s" % (entity.lower(), processed_facts_str, section)
#     pre_string = "<H> %s %s " % (entity.lower(), processed_facts_str)
    return "generate %s : %s" % (languages_map[language]['label'].lower(), pre_string)



def process_facts_shiva(facts, entity):
    """ linearizes the facts on the encoder side """
    triples = []
    subject = entity
    # triples.append([' <S> '+subject])

    for triple in facts:
        predicate = triple[0]
        object = triple[1]
        triples.append([' S| '+subject, ' P| '+predicate, ' O| '+object])

        for quals in triple[2]:
            qual_subject = object
            qual_predicate = quals[0]
            qual_object = quals[1]
            triples.append([' S| '+qual_subject, ' P| '+qual_predicate, ' O| '+qual_object])

    return triples


In [ ]:
cuda0 = torch.device(device)

In [9]:
def collate(batch):
    
    fact_str_list = []
    sent_list = []
    ref_list = []
    lang_list = []
    input_ids_list = []
    attention_mask_list = []
    labels_list = []
    role_ids_list = []
    #'role_ids': role_ids,'labels': labels.squeeze(), 'lang': torch.tensor(lang_id), 'fact_str': fact_str, 'sent': target_text, 'ref_order': ref_order
    ref_flag = True 
    
    try :
        k = batch[0]['ref_order']
    except:
        ref_flag = False
    
    for i in batch :
        fact_str_list.append(i['fact_str'])
        sent_list.append(i['sent'])
        lang_list.append(i['lang'])
        input_ids_list.append(i['input_ids'])
        attention_mask_list.append(i['attention_mask'])
        labels_list.append(i['labels'])
        role_ids_list.append(i['role_ids'])
        
        if ref_flag :
            ref_list.append(i['ref_order'])
        
    
    if ref_flag :
        return {'fact_str':fact_str_list,'sent' : sent_list,'ref_order':ref_list, 'lang': lang_list, 'input_ids': input_ids_list, 'attention_mask': attention_mask_list, 'role_ids': role_ids_list, 'labels': labels_list}
    else:
        return {'fact_str':fact_str_list,'sent' : sent_list, 'lang': lang_list}

In [10]:
class DS(Dataset):
    def __init__(self, data_path, tokenizer, max_source_length, max_target_length):
        # self.df = pd.read_csv(data_path, sep='\t', )
        self.df = jsonl_data
        # self.df = self.df[:len(self.df)]
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        self.languages_map = {
            'en': {"label": "English", 'id': 0},
            'hi': {"label": "Hindi", 'id': 1},
            'te': {"label": "Telugu", 'id': 2}, 
            'bn': {"label": "Bengali", 'id': 3},
            'pa': {"label": "Punjabi", 'id': 4},
            'ur': {"label": "Urdu", 'id': 5}, 
            'or': {"label": "Odia", 'id': 6}, 
            'as': {"label": "Assamese", 'id': 7},
            'gu': {"label": "Gujarati", 'id': 8},
            'mr': {"label": "Marathi", 'id': 9},
            'kn': {"label": "Kannada", 'id': 10},
            'ta': {"label": "Tamil", 'id': 11},
            'ml': {"label": "Malayalam", 'id': 12} 
        }

    def __len__(self):
        return len(self.df)

    def process_facts_shiva(self, facts, entity):
        """ linearizes the facts on the encoder side """
        triples = []
        subject = entity
        # print(facts)

        # triples.append([' <S> '+subject])

        for triple in facts:
            predicate = triple[0]
            object = triple[1]
            triples.append([' S| '+subject, ' P| '+predicate, ' O| '+object])

            for quals in triple[2]:
                qual_subject = object
                qual_predicate = quals[0]
                qual_object = quals[1]
                triples.append([' S| '+qual_subject, ' P| '+qual_predicate, ' O| '+qual_object])

        return triples

    def __getitem__(self, idx):
        # print(type(self.df.iloc[idx]['facts']))
        # input_text = self.process_facts_shiva(eval(self.df.iloc[idx]['facts']))
        entity = self.df.iloc[idx]['entity_name']
        input_text = self.process_facts_shiva(self.df.iloc[idx]['facts'], entity)
        # print(input_text, type(input_text))
        target_text = self.df.iloc[idx]['sentence']
        lang_code = self.df.iloc[idx]['lang']
        lang = self.languages_map[lang_code]['label']
        lang_id = self.languages_map[lang_code]['id']
        prefix = f'rdf to {lang} text: '
        # prefix = f'rdf to en text: '
        try:
            ref_order = self.df.iloc[idx]['order']
        except:
            print("ref_order_not _found")

        input_encoding = self.role_specific_encoding(prefix, input_text)
        # input_encoding = self.tokenizer(input_text, return_tensors='pt', max_length=self.max_source_length ,padding='max_length', truncation=True)
        target_encoding = self.tokenizer(target_text, return_tensors='pt', max_length=self.max_target_length ,padding='max_length', truncation=True)

        input_ids, attention_mask, role_ids, fact_str = input_encoding['input_ids'], input_encoding['attention_mask'], input_encoding['role_ids'], input_encoding['fact_str']
        labels = target_encoding['input_ids']
        labels[labels == self.tokenizer.pad_token_id] = -100    # for ignoring the cross-entropy loss at padding locations

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'role_ids': role_ids,'labels': labels.squeeze(), 'lang': torch.tensor(lang_id), 'fact_str': fact_str, 'sent': target_text, 'ref_order': ref_order}   
        # squeeze() is needed to remove the batch dimension

    def role_specific_encoding(self, prefix, input_text):
        input_ids = []
        attention_mask = []
        role_ids = []
        fact_str = ''

        prefix_tokenized = self.tokenizer.encode(prefix)[:-1]   # ignoring the eos token at the end
        input_ids.extend(prefix_tokenized)
        attention_mask.extend([1] * len(prefix_tokenized))
        role_ids.extend([0] * len(prefix_tokenized))
        fact_str += prefix
        try:
            # data = json.loads(input_text)
            # data = eval(input_text)
            data = input_text
        except json.decoder.JSONDecodeError:
            print(input_text)
            raise

        for triple in data:
            subject = triple[0]
            predicate = triple[1]
            object = triple[2]

            subject_tokenized = self.tokenizer.encode(subject)[:-1]
            predicate_tokenized = self.tokenizer.encode(predicate)[:-1]
            object_tokenized = self.tokenizer.encode(object)[:-1]

            input_ids.extend(subject_tokenized)
            attention_mask.extend([1] * len(subject_tokenized))
            role_ids.extend([1] * len(subject_tokenized))

            input_ids.extend(predicate_tokenized)
            attention_mask.extend([1] * len(predicate_tokenized))
            role_ids.extend([2] * len(predicate_tokenized))

            input_ids.extend(object_tokenized)
            attention_mask.extend([1] * len(object_tokenized))
            role_ids.extend([3] * len(object_tokenized))

            fact_str += subject + predicate + object
        
        input_ids.extend([self.tokenizer.eos_token_id])
        input_ids = self.pad_and_truncate(input_ids)
        attention_mask = self.pad_and_truncate(attention_mask)
        role_ids = self.pad_and_truncate(role_ids)

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'role_ids': role_ids, 'fact_str': fact_str}
    
    def pad_and_truncate(self, ids):
        if len(ids) > self.max_source_length:
            return torch.tensor(ids[:self.max_source_length])
        else:
            return torch.tensor(ids + [self.tokenizer.pad_token_id] * (self.max_source_length - len(ids)))
            # above line is hacky, i.e. padding with pad_token_id, but it works for now. for attention_mask, it should be 0.

    @staticmethod
    def create_tokenizer(tokenizer_name_or_path):
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)
        tokenizer.add_tokens(['S|', 'P|', 'O|'])
        print("we added S|, P|, O| to the tokenizer")
        return tokenizer


In [11]:
ds = DS('en_order.csv', tokenizer, 384, 128)

In [13]:
ds[0]

{'input_ids': tensor([   259,  62458,    288,   5413,   7461,    267,    259,    259, 250100,
          58644,   1515,   6485,    259, 250101,   5255,    304,  61324,    259,
         250102,    644,    259,   3829,  52570,    259, 250100,  58644,   1515,
           6485,    259, 250101,    259,  96781,    259, 250102,  31729,    321,
            259, 250100,  58644,   1515,   6485,    259, 250101,  11395,    304,
            259, 102207,  10260,    259, 250102,  24272,      1,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0

In [12]:
dl = DataLoader(ds, batch_size=4, num_workers=0,shuffle=False, collate_fn=collate)

In [13]:
model.device

device(type='cpu')

In [ ]:
from numpy.linalg import norm
from numpy import dot

BATCH_SIZE = 32
from tqdm import tqdm

ref_order = []
pred_order = []

# data_set =JsonL(jsonl_data,tokenizer,518)
# load_params = {"batch_size": BATCH_SIZE, "shuffle": False, "num_workers": 0}
# data_loader = DataLoader(data_set, **load_params, collate_fn = collate)


for _, data in enumerate(dl, 0):
#     print (data)
    
    # sent = data['sent']
    # fact_str = data['fact_str']
    # lang = data['lang']
    # ref_order += data['ref_order']

    # print(data)
    # break
    
    # print ("sent ",len(sent))
    # print ("fact ",len(fact_str))

# #     print (sent)
#     encoded_sentence = tokenizer.batch_encode_plus(sent, return_tensors="pt",padding=True)
#     encoded_facts = tokenizer.batch_encode_plus(fact_str, return_tensors="pt",padding=True)
    
    model.to(device)
    print("model shifted to device")
    with torch.no_grad():
        output = model(
            input_ids=torch.stack(data["input_ids"]).to(device),
            attention_mask=torch.stack(data["attention_mask"]).to(device,dtype=torch.long),
            role_ids=torch.stack(data["role_ids"]).to(device,dtype=torch.long),
            decoder_input_ids=torch.stack(data["labels"]).to(device,dtype=torch.long),
            output_attentions=True  
        )
    # with torch.no_grad():
    #     output = model(
    #         input_ids=torch.stack(data["input_ids"]),
    #         attention_mask=torch.stack(data["attention_mask"]),
    #         role_ids=torch.stack(data["role_ids"]),
    #         decoder_input_ids=torch.stack(data["labels"]),
    #         output_attentions=True  
    #     )
    # temp = torch.stack(data['input_ids']).to(device)
    # print(temp)

    # input_ids = data['labels']
    # for i in range(4):
    #     print(max(input_ids[0]))
    # break
    
# #     print (output.cross_attentions)
#     print (output.cross_attentions[-1].shape)
#     batch_fin_crossattention = output.cross_attentions[-1].mean(dim=1)
# #     batch_fin_crossattention = output.cross_attentions[-1][:,2,:,:]
#     print (batch_fin_crossattention.shape)
#     print ("batch ",len(batch_fin_crossattention))
    
#     for itr in range(len(batch_fin_crossattention)):
        
#         fin_crossattention = batch_fin_crossattention[itr]
#         print (fin_crossattention.shape)
    
#         stflag = -1
#         currfact = -1
#         factpos = []
#         factpos2 = []
        
#         tokenized_sent = tokenizer.tokenize(sent[itr])
#         tokenized_fact = tokenizer.tokenize(fact_str[itr])
#         sent_len = len(tokenized_sent)
#         for wordval in range(len(tokenized_fact)):

#             tokenized_curr = tokenizer.tokenize(fact_str[itr])[wordval]
#             if (tokenized_curr) == "<R>" and stflag == -1:
#                 stflag = 1

#                 sumval = 0 
#                 fcount = 0
#                 maxpos = 0
#                 maxscore = -99
#                 print ("FIRST FACT")

#                 continue

#             elif (tokenized_curr) == "<R>" and stflag != -1:
#                 factpos.append(maxpos)
#                 factpos2.append(sumval/fcount)

#                 stflag = 1

#                 sumval = 0 
#                 fcount = 0
#                 maxpos = 0
#                 maxscore = -99
#                 print (len(factpos) , " FACT OVER")

#                 continue 


#             elif (tokenized_curr) == "<QR>":
#     #             stflag = 0
#                 print ("QR FOUNDS")
#                 continue

#             elif (tokenized_curr) == "<S>":
#                 factpos.append(maxpos)
#                 factpos2.append(sumval/fcount)
#                 stflag = 0
#                 print (len(factpos) , "LAST FACT OVER")
#                 continue


# #             print(fin_crossattention[:,wordval][:sent_len])
#             max_fact = fin_crossattention[:,wordval][:sent_len].argmax()
#             argval = fin_crossattention[:,wordval][:sent_len].max()
#             try:
#                 print ("fact subword : ",tokenized_fact[wordval],end = "\t\t")
#             except :
#                 print ("hello")
#                 pass
            
#             intval = np.array(max_fact.to('cpu'))
#             tempind = encoded_facts.word_ids()[intval]
#             print ("matched sentence sub word : ",tokenized_sent[intval])

#             if stflag == 1:
#                 sumval += (intval+1)
#                 fcount += 1
#                 if argval > maxscore:
#                     maxpos = (intval+1)
#                     maxscore = argval


#         print ("===end===")
#         final_ordering =  np.argsort(factpos)
#         pred_order.append(list(final_ordering))


#         print ()
#         print ("works")

    

In [27]:
#ignore when gpu available
layers = [-4, -3, -2, -1] 

from numpy.linalg import norm
from numpy import dot

from tqdm import tqdm

ref_order = []
pred_order = []

count = -1

for obj in tqdm(reader) :
    
    count += 1
    
    #print (obj)
    sent  = obj['sentence']
    factlist = obj['facts']
    ref_order.append(obj['order'])
    entity = obj['entity_name']
    section = obj['native_sentence_section']
    
    #sent_vec = get_hidden_states(sent, tokenizer, model, layers)
    #print (sent)
    #print (len(sent_vec))
    
    print(len(factlist))
    
    scorelist  = []
    
    fact_str = process_facts(factlist,entity,section,lang)
    
    encoded_sentence = tokenizer.encode_plus(sent, return_tensors="pt")
    encoded_facts = tokenizer.encode_plus(fact_str, return_tensors="pt")
    
    with torch.no_grad():
        output = model(input_ids=encoded_facts["input_ids"], decoder_input_ids=encoded_sentence["input_ids"],output_attentions=True)
    
#     print (len(encoded_sentence.word_ids()))
#     print (encoded_sentence.word_ids())
#     print (tokenizer.tokenize(sent))
#     print (tokenizer.tokenize(fact_str))


    print("-"*30)
    print(len(output.cross_attentions))
    print(len(output.cross_attentions[-1]))
    print("-"*30)    
    print (output.cross_attentions[-1][0].shape)
    
    #using mean pooling to resolve multihead attention -> can switch to max pooling as well 
    fin_crossattention = output.cross_attentions[-1][0].max(dim=0).values
    
    which_fact = [] # represents the maximum alligned facts corresponding to each word of the sentence 
    
    print (len(fin_crossattention))
    print (fin_crossattention.shape)
    
#     for wordval in range(len(fin_crossattention)):
#         #print (fin_crossattention[wordval])
#         max_fact = fin_crossattention[wordval][:-1].argmax()
#         try:
#             print ("sentence subword : ",tokenizer.tokenize(sent)[wordval],end = "\t\t")
#         except :
#             pass
#         #print (np.array(max_fact))
#         intval = np.array(max_fact)
#         tempind = encoded_facts.word_ids()[intval]
#         print ("matched fact sub word : ",tokenizer.tokenize(fact_str)[intval])
    
    
    stflag = -1
    currfact = -1
    factpos = []
    factpos2 = []
    
    for wordval in range(len(fin_crossattention[0])-1):
        
        if (tokenizer.tokenize(fact_str)[wordval]) == "<R>" and stflag == -1:
            stflag = 1
            
            sumval = 0 
            fcount = 0
            maxpos = 0
            maxscore = -99
            print ("FIRST FACT")
            
            continue
            
        elif (tokenizer.tokenize(fact_str)[wordval]) == "<R>" and stflag != -1:
            factpos.append(maxpos)
            factpos2.append(sumval/fcount)
            
            stflag = 1
            
            sumval = 0 
            fcount = 0
            maxpos = 0
            maxscore = -99
            print (len(factpos) , " FACT OVER")
            
            continue 
            
            
        elif (tokenizer.tokenize(fact_str)[wordval]) == "<QR>":
#             stflag = 0
            print ("QR FOUNDS")
            continue
            
        elif (tokenizer.tokenize(fact_str)[wordval]) == "<S>":
            factpos.append(maxpos)
            factpos2.append(sumval/fcount)
            stflag = 0
            print (len(factpos) , "LAST FACT OVER")
            continue
        
            
        max_fact = fin_crossattention[:,wordval][:-1].argmax()
        argval = fin_crossattention[:,wordval][:-1].max()
        try:
            print ("fact subword : ",tokenizer.tokenize(fact_str)[wordval],end = "\t\t")
        except :
            print ("hello")
            pass
        intval = np.array(max_fact)
        tempind = encoded_facts.word_ids()[intval]
        print ("matched sentence sub word : ",tokenizer.tokenize(sent)[intval])
        
        if stflag == 1:
            sumval += (intval+1)
            fcount += 1
            if argval > maxscore:
                maxpos = (intval+1)
                maxscore = argval
        
        
    print ("===end===")
    final_ordering =  np.argsort(factpos2)
    pred_order.append(list(final_ordering))
    

  0%|          | 0/100 [00:00<?, ?it/s]

2


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking arugment for argument index in method wrapper_index_select)

In [16]:
import random

qidx = random.choice([z for z in range(100)])
print(pred_order[qidx])
print(ref_order[qidx])

[1, 0]
[1, 0]


In [17]:
def get_accuracy(a, b):
    assert len(a)==len(b), "unequal size of list"
    count = 0
    for y, z in zip(a, b):
        if y==z:
            count+=1
    return count/len(a)

In [18]:
from scipy.stats import kendalltau, spearmanr
import numpy as np

In [19]:
avg_spearman = np.array([spearmanr(y, z)[0] for y, z in zip(pred_order, ref_order)])
avg_kendalltau = np.array([kendalltau(y, z)[0] for y, z in zip(pred_order, ref_order)])

In [20]:
print("spearman", avg_spearman.mean())
print("kendalltau", avg_kendalltau.mean())
print("accuracy", get_accuracy(ref_order, pred_order))

spearman 0.18257142857142852
kendalltau 0.17066666666666666
accuracy 0.38


In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel

/home2/shivprasad.sagare/miniconda3/envs/xalign_role/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained('google/mt5-small')
tokenizer.add_tokens(['S|', 'P|', 'O|'])

3

In [4]:
tokenizer.tokenize(' ')

['▁']